### Install Non Built-in Libraries

In [1]:
# uncomment the following line code if nbimporter library not installed
# !pip install nbimporter

# uncomment the following line code if pycountry library not installed
# !conda install -c conda-forge pycountry

### Import Libraries

In [2]:
# to import from other notebooks
import nbimporter

# to convert countries alpha2 code to country name
import pycountry

# for deployment
import flask   
import pickle
import json
from flask import Flask, request, Response, jsonify, render_template

### Load Model & Training Instants 

In [3]:
# model pipeline
model = pickle.load(open('models/svm_pipeline.pkl', 'rb'))

# label encoder instant
le = pickle.load(open('models/labelencoder.pkl', 'rb'))

### Text Preprocessing 

In [4]:
# import preprocessing function from 
from Data_Preprocessing import *

In [5]:
# create specific functions

## remove_punctuations
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ»«•'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    for punc in punctuations_list:
        text = text.replace(punc, '')
    return text

## remove_diacritics
arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def remove_diacritics(text):
    return re.sub(arabic_diacritics, '', text)


In [6]:
# create function 
preprocessing_functions = [remove_newlines_tabs,remove_tages_hashtags, remove_emojis,
                           remove_links, remove_punctuations, remove_diacritics,
                           remove_numbers, remove_english, remove_elongation,
                           normalize_text, remove_whitespaces]


def text_preprocessing(text):
    for func in preprocessing_functions:
        text = func(text)
    return text

In [7]:
# countries dictionary
country_dict = {}

# extract only countries we deal with in doalect case
for country in pycountry.countries:
    if country.alpha_2 in le.classes_:
        country_dict[country.alpha_2] = country.name

# correct wrong country name
country_dict['PL'] = 'Palestine'

### ReSTful API Web Application
- get request

In [8]:
# design the web service
app = Flask(__name__)


# home route
@app.route('/')
def home():
    return '<br><br><h1 style="text-align:center">Welcome To Dialect Prediction Model</h1>'


# prediction route
@app.route('/get_predict/', methods=['GET'])
def get_predict():
    
    # get text and preprocess
    originial_text = str(request.args['text'])
    text = text_preprocessing(originial_text)
    
    # validation step
    if len(text) == 0:
        return Response(json.dumps({'error': 'Not Sufficient Text.'}), status=500)
    
    else:
        # make a prediction
        model_pred = model.predict([text])
        dialect_class = le.inverse_transform([model_pred])[0]
        country_name = country_dict[dialect_class]
    
    # retun json
    return jsonify(text = str(originial_text), dialect = dialect_class, country = country_name)

In [9]:
# run server
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2022 15:02:19] "GET /get_predict?text=عايزين+حاجات+من+السوبرماركت HTTP/1.1" 308 -
C:\Users\Wssam\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
127.0.0.1 - - [14/Mar/2022 15:02:19] "GET /get_predict/?text=عايزين+حاجات+من+السوبرماركت HTTP/1.1" 200 -


### GUI Web Application

In [10]:
# design the web service
app = Flask(__name__)


# home route
@app.route('/home', methods=['GET', 'POST'])
@app.route('/', methods=['GET'])
def main():
    return render_template('mainpage.html')


# prediction route
@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        
        # get text from post request and preprocess
        text = request.form['text']
        text = text_preprocessing(text)

        # validation step
        if len(text) == 0:
            return render_template('predictionpage.html', dialect = '', country = 'Not Sufficient Text')
        
        else:
            # make a prediction
            model_pred = model.predict([text])
            dialect_class = le.inverse_transform([model_pred])[0]
            country_name = country_dict[dialect_class]
        
    return render_template('predictionpage.html', dialect = dialect_class, country = country_name)

In [11]:
# run server
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2022 15:02:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 15:02:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2022 15:03:13] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 15:03:36] "POST /home HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 15:03:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2022 15:03:41] "POST /predict HTTP/1.1" 200 -


### Done!